# Reading a 2D obj file
Reading a 2D .obj file with material IDs and boundary IDs.

Click here for more information on [WaveFront objects,
    aka obj](https://en.wikipedia.org/wiki/Wavefront_.obj_file)

To run the code, simply type: `jupyter nbconvert --to python --execute <basename>.ipynb`.

To convert it to a python file (named `<basename>.py`), simply type: `jupyter nbconvert --to python <basename>.ipynb`

To run the python file from the terminal, using N processes, simply type: `mpiexec -n <N> python <basename>.py`

In [ ]:
import os
import sys
from mpi4py import MPI

sys.path.append("../..")

from pyopensn.mesh import FromFileMeshGenerator, PETScGraphPartitioner
from pyopensn.context import UseColor, Finalize

UseColor(False)

## Mesh and  Partition
We have created an unstructured triangular mesh with the
[Triangle Mesh Generator](http://www.cs.cmu.edu/~quake/triangle.html).
Triangle is also accessible via [Meshpy](https://documen.tician.de/meshpy/) in python.
Then, we converted the data generated by Triangle into the Wavefront (obj) format.

We use the ```FromFileMeshGenerator``` and pass the path to the obj file.

We also partition the 2D mesh into 4 subdomains using `Parmetis` (by running using `mpiexec -n 4 ...`).

Finally, we export the mesh to a VTU file.

In [ ]:
meshgen = FromFileMeshGenerator(
    filename="../../../../test/assets/mesh/GMSH_AllTets.vtu",
    partitioner=PETScGraphPartitioner(type='parmetis'),
)
grid = meshgen.Execute()

# Export
grid.ExportToPVTU("Read_3D_vtu")

## Visualization

The resulting mesh and material layout is shown below:

![Mesh_Material](images/Triangle_1542_mesh_material.png)

When using the Parmetis partitioner and 4 MPI ranks, we obtain:

![Mesh_Partition](images/Triangle_1542_partition_Parmetis.png)

FYI, had we chosen the KBA partitioner, the partition would have been:

![Mesh_Partition](images/Triangle_1542_partition_KBA.png)


## Finalize (for Jupyter Notebook only)

In Python script mode, PyOpenSn automatically handles environment termination. However, this
automatic finalization does not occur when running in a Jupyter notebook, so explicit finalization
of the environment at the end of the notebook is required. Do not call the finalization in Python
script mode, or in console mode.

Note that PyOpenSn's finalization must be called before MPI's finalization.


In [ ]:
from IPython import get_ipython

def finalize_env():
    Finalize()
    MPI.Finalize()

ipython_instance = get_ipython()
if ipython_instance is not None:
    ipython_instance.events.register("post_execute", finalize_env)